In [ ]:
import pandas as pd
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI
df = pd.read_csv("/Users/matansharon/python/chat_with_docs/data/csv/banklist.csv",encoding='Windows-1252')

engine = create_engine("sqlite:///banklist.db")
df.to_sql("banklist", engine, index=False)

In [ ]:

db = SQLDatabase(engine=engine)
gpt3 = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

gpt4 = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0)
gpt3_agent_executor = create_sql_agent(gpt3, db=db, agent_type="openai-tools", verbose=True)
gpt4_agent_executor = create_sql_agent(gpt3, db=db, agent_type="openai-tools", verbose=True)

In [ ]:
gpt3_agent_executor.invoke({"input": "how many diffrence cities there is in the dataset?"})['output']

In [ ]:
gpt4_agent_executor.invoke({"input": "how many diffrence cities there is in the dataset?"})['output']

In [ ]:
gpt4_agent_executor.invoke({"input": "what is the cert column mean?"})['output']